## Install (run once)

In [51]:
!git clone https://github.com/jldbc/pybaseball
%cd pybaseball
!pip install -e .

from pybaseball import statcast_batter, playerid_lookup
from pybaseball import statcast
from pybaseball import cache
import pandas as pd

!pip3 install plotly
import plotly.express as px

cache.enable()

fatal: destination path 'pybaseball' already exists and is not an empty directory.
/Users/benzene/Documents/personal_code/pybaseball/ops++/pybaseball/pybaseball
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /Users/benzene/Documents/personal_code/pybaseball/ops++/pybaseball/pybaseball
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


## Collect data league wide

In [2]:
data_2022 = statcast(start_dt='2022-04-06', end_dt='2022-12-31')

This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 224/224 [03:13<00:00,  1.16it/s]


In [4]:
# Change many differnt outs to just be an out
outs = ['field_out', 'force_out', 'grounded_into_double_play', 'fielders_choice_out', 
        'sac_bunt_double_play', 'fielders_choice', 'double_play', 'triple_play', 
        'sac_fly_double_play', 'strikeout_double_play', 'strikeout', 'other_out']

data_2022.loc[data_2022['events'].isin(outs), 'events'] = 'out'
care_about = ['hit_by_pitch', 'out', 'walk', 'home_run', 'double', 'sac_fly', 'triple', 'single']
data_2022 = data_2022.loc[data_2022['events'].isin(care_about)]

In [5]:
data_2022.describe()

,release_speed,release_pos_x,release_pos_z,batter,pitcher,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,...,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp
count,182505.0,182509.0,182509.0,183039.0,183039.0,0.0,0.0,0.0,0.0,182509.0,...,183039.0,183039.0,183039.0,183039.0,183039.0,183039.0,183039.0,182018.0,183039.0,182966.0
mean,89.016052,-0.809914,5.796464,614750.219931,613272.238944,<NA>,<NA>,<NA>,<NA>,7.945247,...,2.20577,2.119576,2.148553,2.26148,2.118947,2.231874,2.148553,176.183152,0.000425,0.007175
std,6.141273,1.827518,0.568952,59732.767003,61559.290231,<NA>,<NA>,<NA>,<NA>,4.078035,...,2.575636,2.485302,2.561739,2.598772,2.494656,2.533991,2.561739,71.40087,0.05521,0.458248
min,32.7,-4.95,0.87,405395.0,405395.0,<NA>,<NA>,<NA>,<NA>,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.736,-1.473
25%,84.8,-2.08,5.52,592206.0,592789.0,<NA>,<NA>,<NA>,<NA>,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131.0,-0.022,-0.252
50%,89.9,-1.5,5.84,640458.0,623474.0,<NA>,<NA>,<NA>,<NA>,8.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,200.0,0.0,-0.141
75%,93.9,0.88,6.14,664034.0,663556.0,<NA>,<NA>,<NA>,<NA>,12.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,223.0,0.022,0.203
max,104.2,4.75,7.51,691910.0,801389.0,<NA>,<NA>,<NA>,<NA>,14.0,...,28.0,28.0,28.0,28.0,21.0,28.0,28.0,360.0,0.91,3.605


In [6]:
# Check to make sure that we have the right types of events
data_2022['events'].unique()

array(['out', 'single', 'walk', 'home_run', 'hit_by_pitch', 'double',
       'sac_fly', 'triple'], dtype=object)

In [72]:
def make_ops(df):
    teams = df['home_team'].unique()

    player_ops = 0
    total_abs = len(df)

    df = df.groupby(['home_team', 'events']).count()
    df = df.add_suffix('_Count').reset_index()
    df = df[['home_team', 'events', 'away_team_Count']]

    # Taken from https://baseballsavant.mlb.com/leaderboard/statcast-park-factors
    park_factor = {
        'ATL': 100,
        'AZ': 101,
        'BAL': 101,
        'BOS': 109,
        'CHC': 98,
        'CIN': 111,
        'CLE': 99,
        'COL': 112,
        'CWS': 102,
        'DET': 97,
        'HOU': 98,
        'KC': 103,
        'LAA': 104,
        'LAD': 102,
        'MIA': 99,
        'MIL': 100,
        'MIN': 98,
        'NYM': 96,
        'NYY': 99,
        'OAK':94,
        'PHI': 103,
        'PIT': 100,
        'SD': 94, 
        'SEA': 91,
        'SF': 100,
        'STL': 95,
        'TB': 96,
        'TEX': 99,
        'TOR': 99,
        'WSH': 100
    }

    for i in teams:
        team_df = df.loc[df['home_team'] == i]
        AB = team_df['away_team_Count'].sum()
        team_df = team_df.pivot_table(index=['home_team'], 
                        columns='events', 
                        values='away_team_Count')
        
        single = int(team_df['single']) if 'single' in team_df else 0
        double = int(team_df['double']) if 'double' in team_df else 0
        triple = int(team_df['triple']) if 'triple' in team_df else 0
        home_run = int(team_df['home_run']) if 'home_run' in team_df else 0

        H = single + double + triple + home_run
        BB = int(team_df['walk']) if 'walk' in team_df else 0
        HBP = int(team_df['hit_by_pitch']) if 'hit_by_pitch' in team_df else 0
        SF = int(team_df['sac_fly']) if 'sac_fly' in team_df else 0
        TB = single + (2 * double) + (3 * triple) + (4 * home_run)

        AB = AB - BB - HBP - SF
        if(AB != 0):
            ops_team = (((H + BB + HBP) / (AB + BB + SF + HBP)) + (TB/AB)) * (100/park_factor[i])
            player_ops += (ops_team) * (AB/total_abs)
            
    return player_ops

In [74]:
# Find average ops this will help to normalize our ops++
ave_ops = make_ops(data_2022)
print('ave_ops: {}'.format(ave_ops))


ave_ops: 0.6387842369057117


## Data for single player

In [75]:
def make_ops_pp(df, ave_ops):
    player_ops = make_ops(df)
            
    return round((player_ops/ave_ops)*100)

In [76]:
first_name = 'Mookie'
last_name = 'Betts'

player = playerid_lookup(last_name, first_name)
if(len(player) > 1):
    player = player.iloc[len(player)-1]

data_2022_player = statcast_batter(start_dt='2022-04-06', end_dt='2022-12-31', player_id = int(player['key_mlbam']))

outs = ['field_out', 'force_out', 'grounded_into_double_play', 'fielders_choice_out', 
        'sac_bunt_double_play', 'fielders_choice', 'double_play', 'triple_play', 
        'sac_fly_double_play', 'strikeout_double_play', 'strikeout', 'other_out']

data_2022_player.loc[data_2022_player['events'].isin(outs), 'events'] = 'out'
care_about = ['hit_by_pitch', 'out', 'walk', 'home_run', 'double', 'sac_fly', 'triple', 'single']
data_2022_player = data_2022_player.loc[data_2022_player['events'].isin(care_about)]

data_2022_player

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2022-10-15,92.0,-1.29,6.39,"Betts, Mookie",605141,605397,out,hit_into_play,...,0,2,0,2,0,Infield shift,Standard,205.0,0.017,-0.221
1,FF,2022-10-15,94.2,-1.26,6.38,"Betts, Mookie",605141,605397,walk,ball,...,0,0,0,0,0,Infield shift,Standard,209.0,-0.028,0.212
7,SL,2022-10-15,81.9,-2.12,5.33,"Betts, Mookie",605141,621051,walk,ball,...,0,2,0,2,0,Infield shift,Standard,76.0,-0.023,0.321
9,FF,2022-10-15,95.2,-1.28,6.35,"Betts, Mookie",605141,605397,out,called_strike,...,0,0,0,0,0,Infield shift,Standard,213.0,0.022,-0.195
13,SL,2022-10-15,85.8,3.30,5.24,"Betts, Mookie",605141,623352,out,swinging_strike,...,5,3,5,3,5,Standard,Standard,264.0,0.037,-0.141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,CH,2022-04-08,87.1,1.70,5.56,"Betts, Mookie",605141,607536,double,hit_into_play,...,2,3,2,3,2,Standard,Standard,138.0,-0.145,0.706
2482,SI,2022-04-08,91.8,1.70,5.49,"Betts, Mookie",605141,607536,out,foul_tip,...,2,0,2,0,2,Standard,Standard,142.0,0.038,-0.251
2486,ST,2022-04-08,80.7,-3.44,4.92,"Betts, Mookie",605141,664875,out,hit_into_play,...,2,5,2,5,2,None,None,86.0,0.004,-0.132
2487,KC,2022-04-08,82.6,1.86,5.44,"Betts, Mookie",605141,607536,out,swinging_strike,...,0,0,0,0,0,Infield shift,Standard,321.0,-0.036,0.459


In [77]:
player_ops = make_ops_pp(data_2022_player, ave_ops)
print('{} {}: {}'.format(first_name, last_name, player_ops))

Mookie Betts: 120


## Data for all players

In [87]:
def progbar(curr, total, full_progbar):
    frac = curr/total
    filled_progbar = round(frac*full_progbar)
    print('\r', '#'*filled_progbar + '-'*(full_progbar-filled_progbar), '[{:>7.2%}]'.format(frac), end='')

table = []
batters = data_2022['batter'].unique()
for i, batter in enumerate(batters):
    df_batter = data_2022.loc[data_2022['batter'] == batter]
    player_ops = make_ops_pp(df_batter, ave_ops)
    table.append([batter, player_ops, len(df_batter)])
    progbar(i+1, len(batters),20)

ops_pp_df = pd.DataFrame(table, columns=['batter', 'ops_pp', 'ABs'])
ops_pp_df

 #################### [100.00%]

,batter,ops_pp,ABs
0,592206,99,625
1,547180,129,485
2,592663,112,631
3,656555,107,744
4,663656,113,660
...,...,...,...
700,683610,163,2
701,680858,0,1
702,680579,0,1
703,691910,0,1


In [96]:
df = px.data.tips()
fig = px.histogram(ops_pp_df, x='ops_pp', title='ops++ Histogram 2022')
fig.show()

In [91]:
ops_pp_df[ops_pp_df['ops_pp'] > 150]

,batter,ops_pp,ABs
152,672640,192,10
551,644433,316,4
562,681546,171,16
635,666137,376,2
652,656388,157,11
655,542963,158,3
695,682927,163,2
698,678860,163,2
700,683610,163,2


It seems that all of our super high ops++ are very small sample size!

In [92]:
ops_pp_df_clean = ops_pp_df[ops_pp_df['ABs'] >= 50]
ops_pp_df_clean

,batter,ops_pp,ABs
0,592206,99,625
1,547180,129,485
2,592663,112,631
3,656555,107,744
4,663656,113,660
...,...,...,...
661,605113,110,53
666,657061,61,63
667,641856,46,56
668,547379,95,69


In [98]:
df = px.data.tips()
fig = px.histogram(ops_pp_df_clean, x='ops_pp', title='ops++ Histogram 2022 (Above 50 Plate Appearances')
fig.show()

This looks roughly normal so we can make percentiles from their ops++ if we wanted too!